# Lab 5: Single Agent with MCP Server Integration

Build an AI agent using **Microsoft Agent Framework** that connects to your **MCP servers** from Lab 4!

## Learning Objectives

By the end of this lab, you will:
1. Connect an Azure AI Agent to a **local MCP server**
2. Connect an Azure AI Agent to a **remote (deployed) MCP server**
3. Understand the difference between **HostedMCPTool** vs **MCPStreamableHTTPTool**
4. Implement **user approval workflows** for secure tool execution
5. Build a complete **Gradio chat interface** with MCP-powered tools

## Prerequisites

| Requirement | Setup |
|------------|-------|
| Python environment | Run `uv sync --all-extras --dev` from project root |
| Azure CLI logged in | Run `az login` in terminal |
| Azure AI Foundry Project | [Create at ai.azure.com](https://ai.azure.com) |
| Model deployment | Deploy GPT-4o in your AI Foundry project |
| **Lab 4 completed** | MCP server deployed to Azure |

## Environment Variables

This notebook loads from **two `.env` files**:

### 1. Project Root `.env` (Azure AI)
```bash
# Required for Azure AI Foundry
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
```

### 2. Lab 4 MCP `.env` (`src/tool_registry/mcps/.env`)
```bash
# Created automatically by Lab 4 deployment
MCP_REMOTE_URL=https://func-api-xxxx.azurewebsites.net/runtime/webhooks/mcp
MCP_REMOTE_KEY=your-mcp-extension-key
MCP_LOCAL_URL=http://localhost:7071/runtime/webhooks/mcp
```

> 💡 **Note:** Run Lab 4 first to create the MCP `.env` file with your deployment values.

## Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                      Microsoft Agent Framework                               │
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │                    AzureAIAgentsProvider                            │    │
│  │                                                                     │    │
│  │   ┌─────────┐         ┌──────────────────────────────────┐         │    │
│  │   │  Agent  │────────▶│     MCP Tool Integration         │         │    │
│  │   │  (LLM)  │◀────────│                                  │         │    │
│  │   └────┬────┘         │  ┌────────────────────────────┐  │         │    │
│  │        │              │  │ MCPStreamableHTTPTool      │  │         │    │
│  │        │              │  │ (Run-level, manual connect)│  │         │    │
│  │        │              │  └────────────────────────────┘  │         │    │
│  │        │              │  ┌────────────────────────────┐  │         │    │
│  │        │              │  │ HostedMCPTool              │  │         │    │
│  │        │              │  │ (Agent-level, auto-managed)│  │         │    │
│  │        │              │  └────────────────────────────┘  │         │    │
│  │        │              └──────────────────────────────────┘         │    │
│  └────────┼───────────────────────────────────────────────────────────┘    │
└───────────┼─────────────────────────────────────────────────────────────────┘
            │
            │ SSE + JSON-RPC 2.0 (MCP Protocol)
            │
   ┌────────┴────────┐                          ┌────────────────────┐
   │  Local Server   │                          │  Remote Server     │
   │  (func start)   │                          │  (Azure Functions) │
   │  :7071          │                          │  *.azurewebsites.net│
   └─────────────────┘                          └────────────────────┘
         │                                              │
         ▼                                              ▼
   ┌─────────────────────────────────────────────────────────────────┐
   │                     MCP Tools (from Lab 4)                      │
   │  • hello_mcp      - Health check                                │
   │  • analyze_data   - Statistical analysis                        │
   │  • save_snippet   - Save code snippets                          │
   │  • get_snippet    - Retrieve snippets                           │
   │  • list_snippets  - List all snippets                           │
   └─────────────────────────────────────────────────────────────────┘
```

**Key Difference from Lab 3:**
- Lab 3: Tools defined as **Python functions** (local, Logic App HTTP, Azure Function HTTP)
- Lab 5: Tools discovered from **MCP servers** (standardized protocol, auto-discovery)

## MCP Tool Integration Patterns

| Pattern | Class | Connection | Best For |
|---------|-------|------------|----------|
| **Agent-level** | `HostedMCPTool` | Auto-managed by agent | Simple use, less control |
| **Run-level** | `MCPStreamableHTTPTool` | Manual context manager | More control, dynamic tools |

### Agent-Level (HostedMCPTool)
```python
agent = await provider.create_agent(
    tools=HostedMCPTool(name="my-mcp", url="https://...")
)
async with agent:
    result = await agent.run(query)
```

### Run-Level (MCPStreamableHTTPTool)
```python
async with MCPStreamableHTTPTool(name="my-mcp", url="https://...") as mcp:
    result = await agent.run(query, tools=mcp)
```

## Step 1: Setup and Environment

In [5]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

# Ensure Azure CLI is in PATH (Homebrew installs to /opt/homebrew/bin on Apple Silicon)
homebrew_paths = ["/opt/homebrew/bin", "/usr/local/bin"]
current_path = os.environ.get("PATH", "")
for p in homebrew_paths:
    if p not in current_path:
        os.environ["PATH"] = p + ":" + current_path
        current_path = os.environ["PATH"]

# Verify az is accessible
import shutil

az_path = shutil.which("az")
if az_path:
    print(f"✅ Azure CLI found: {az_path}")
else:
    print("⚠️  Azure CLI not found. Run: brew install azure-cli && az login")

# Load environment variables from project root (.env)
project_root_env = Path("../.env")
if project_root_env.exists():
    load_dotenv(project_root_env)
    print(f"✅ Loaded project .env from {project_root_env.resolve()}")

# Also load from Lab 4 MCP server directory (has MCP_REMOTE_URL and MCP_REMOTE_KEY)
mcp_env = Path("../src/tool_registry/mcps/.env")
if mcp_env.exists():
    load_dotenv(mcp_env, override=True)
    print(f"✅ Loaded MCP .env from {mcp_env.resolve()}")
else:
    print("⚠️  MCP .env not found. Run Lab 4 first to create it.")
    print(f"   Expected at: {mcp_env.resolve()}")

# Verify required env vars for Azure AI
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"\n❌ Missing environment variables: {missing}")
    print("   Please create a .env file in the project root with:")
    print("   AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com")
    print("   AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o")
else:
    print("\n✅ Azure AI Environment configured")
    print(f"   Project: {os.getenv('AZURE_AI_PROJECT_ENDPOINT')[:50]}...")
    print(f"   Model: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')}")

# Load MCP server configuration from Lab 4 .env
# Variable names match what Lab 4 creates: MCP_REMOTE_URL, MCP_REMOTE_KEY, MCP_LOCAL_URL
print("\n🔧 MCP Server Configuration (from Lab 4 .env):")
mcp_remote_url = os.getenv("MCP_REMOTE_URL")
mcp_remote_key = os.getenv("MCP_REMOTE_KEY")
mcp_local_url = os.getenv("MCP_LOCAL_URL", "http://localhost:7071/runtime/webhooks/mcp")

if mcp_remote_url:
    print(f"   ✅ Remote MCP: {mcp_remote_url[:60]}...")
else:
    print("   ⚠️  MCP_REMOTE_URL not set (run Lab 4 to deploy)")

if mcp_remote_key:
    print(f"   ✅ Remote Key: {mcp_remote_key[:20]}...")
else:
    print("   ⚠️  MCP_REMOTE_KEY not set")

print(f"   📍 Local MCP: {mcp_local_url}")

✅ Azure CLI found: /opt/homebrew/bin/az
✅ Loaded project .env from /Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/.env
✅ Loaded MCP .env from /Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/src/tool_registry/mcps/.env

✅ Azure AI Environment configured
   Project: https://aihubnw3478841489.services.ai.azure.com/ap...
   Model: gpt-4o

🔧 MCP Server Configuration (from Lab 4 .env):
   ✅ Remote MCP: https://func-api-4csy4jcpcp47y.azurewebsites.net/runtime/web...
   ✅ Remote Key: F5FQQTVudG4tJ6k7PNYZ...
   📍 Local MCP: http://localhost:7071/runtime/webhooks/mcp


## Step 2: Install Required Packages

The `agent-framework` package provides the MCP integration classes.

In [8]:
# Install agent-framework if not already installed
import subprocess

try:
    from agent_framework import MCPStreamableHTTPTool, HostedMCPTool

    print("✅ agent_framework already installed")
except ImportError:
    print("📦 Installing agent-framework...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "agent-framework", "-q"])
    from agent_framework import MCPStreamableHTTPTool, HostedMCPTool

    print("✅ agent_framework installed successfully")

# Verify imports
from agent_framework import (
    AgentProtocol,
    AgentResponse,
    AgentThread,
    HostedMCPTool,
    MCPStreamableHTTPTool,
)
from agent_framework.azure import AzureAIAgentsProvider
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

print("\n✅ All imports successful:")
print("   • AgentProtocol, AgentResponse, AgentThread")
print("   • HostedMCPTool, MCPStreamableHTTPTool")
print("   • AzureAIAgentsProvider")
print("   • DefaultAzureCredential, AzureCliCredential")

✅ agent_framework already installed

✅ All imports successful:
   • AgentProtocol, AgentResponse, AgentThread
   • HostedMCPTool, MCPStreamableHTTPTool
   • AzureAIAgentsProvider
   • DefaultAzureCredential, AzureCliCredential


## Step 3: Connect to Local MCP Server

First, let's connect to the **local MCP server** you created in Lab 4.

### ⚠️ Before running this cell:
1. Open a terminal in the `src/tool_registry/mcps/src` directory
2. Start Azurite: `azurite --silent &`
3. Start the MCP server: `func start`
4. Verify it's running at http://localhost:7071

In [9]:
import asyncio
from typing import Any

# Use MCPStreamableHTTPTool for run-level tool integration
# This gives us more control over the connection lifecycle


async def test_local_mcp_agent():
    """Test agent with local MCP server (tools defined at run level)."""

    print("🔌 Connecting to local MCP server...")
    print(f"   URL: {mcp_local_url}")
    print()

    async with (
        DefaultAzureCredential() as credential,
        # MCPStreamableHTTPTool connects to the MCP server
        # It auto-discovers available tools via the MCP protocol
        # Note: load_prompts=False because Azure Functions MCP doesn't support prompts
        MCPStreamableHTTPTool(
            name="lab4-local-mcp",
            url=mcp_local_url,
            load_prompts=False,  # Azure Functions MCP doesn't support prompts/list
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create agent WITHOUT tools (we'll pass them at run time)
        agent = await provider.create_agent(
            name="MCPAssistant",
            instructions="""You are a helpful assistant with access to MCP tools.
            
Available MCP Tools (from Lab 4):
- hello_mcp: Health check to verify MCP connectivity
- analyze_data: Analyze numbers and return statistics (count, sum, mean, min, max)
- save_snippet: Save a code snippet with a name
- get_snippet: Retrieve a saved snippet by name
- list_snippets: List all saved snippet names

When asked to analyze data, use the analyze_data tool with JSON array format.
When asked to save/get code, use the snippet tools.""",
        )

        print(f"✅ Agent created: {agent.name}")
        print(f"   MCP Server: lab4-local-mcp")
        print()

        # Test queries
        test_queries = [
            "Say hello using the MCP server",
            "Analyze these numbers: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]",
            "Save this Python code as 'hello_world': print('Hello, Northwestern!')",
            "List all saved snippets",
        ]

        for query in test_queries:
            print("=" * 70)
            print(f"📤 User: {query}")
            print("-" * 70)
            # Pass MCP tools at run time
            result = await agent.run(query, tools=mcp_server)
            print(f"🤖 Assistant: {result}")
            print()


# Run the test
await test_local_mcp_agent()

🔌 Connecting to local MCP server...
   URL: http://localhost:7071/runtime/webhooks/mcp

✅ Agent created: MCPAssistant
   MCP Server: lab4-local-mcp

📤 User: Say hello using the MCP server
----------------------------------------------------------------------
🤖 Assistant: Hello from MCP Server! 👋  
Status: Success  
Lab: Lab 4 - MCP Server with Azure Functions

📤 User: Analyze these numbers: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
----------------------------------------------------------------------
🤖 Assistant: Here are the statistics for the numbers you provided:

- **Count**: 10 (the total number of values)
- **Sum**: 550 (the total of all numbers)
- **Mean**: 55.0 (the average value)
- **Minimum**: 10 (the smallest number)
- **Maximum**: 100 (the largest number)

📤 User: Save this Python code as 'hello_world': print('Hello, Northwestern!')
----------------------------------------------------------------------
🤖 Assistant: The Python code snippet named 'hello_world' has been saved

## Step 4: Connect to Remote MCP Server (Azure)

Now let's connect to the **deployed MCP server** in Azure using `HostedMCPTool`.

### Why HostedMCPTool for Remote Servers?

| Aspect | MCPStreamableHTTPTool | HostedMCPTool |
|--------|----------------------|---------------|
| Connection | **Client-side** (your machine → MCP) | **Server-side** (Azure AI → MCP) |
| Local servers | ✅ Works great | ❌ Azure can't reach localhost |
| Remote servers | ⚠️ Timeout issues | ✅ More reliable |
| Tool approval | Auto-executes | **Requires explicit approval** |

**Key insight**: For **remote** MCP servers, use `HostedMCPTool` because Azure AI Agents service connects to the MCP server directly (server-to-server).

```
┌─────────────────┐     ┌──────────────────────┐     ┌─────────────────┐
│  Your Notebook  │────▶│  Azure AI Agents     │────▶│  MCP Server     │
│  (client)       │     │  (cloud service)     │     │  (Azure Func)   │
└─────────────────┘     └──────────────────────┘     └─────────────────┘
                              HostedMCPTool
                         (server-side connection)
```

### ⚠️ Important: Tool Approval Required

`HostedMCPTool` **requires explicit approval** for each tool call. The response will contain `FunctionApprovalRequestContent` items that must be approved before the tool executes.

```python
# Handle approval loop
while len(result.user_input_requests) > 0:
    approvals = [
        ChatMessage(role="user", contents=[req.create_response(True)])
        for req in result.user_input_requests
    ]
    result = await agent.run(approvals, thread=thread, store=True)
```

### Other Notes
- `HostedMCPTool` name must match pattern `^[a-zA-Z0-9_]+$` (use underscores, NOT hyphens)
- Use `AzureCliCredential` for better compatibility
- Use `async with agent:` to manage MCP connection lifecycle
- **Authentication**: Use `?code=KEY` query parameter

In [23]:
from agent_framework import ChatMessage


async def test_remote_mcp_agent():
    """Test agent with remote (Azure) MCP server using HostedMCPTool.

    HostedMCPTool connects SERVER-SIDE through Azure AI Agents service,
    which is more reliable for remote MCP servers than client-side connections.

    NOTE: HostedMCPTool requires explicit approval for tool calls.
    We handle this with an auto-approval loop.
    """

    if not mcp_remote_url or not mcp_remote_key:
        print("❌ Remote MCP not configured!")
        print("   Set MCP_REMOTE_URL and MCP_REMOTE_KEY in your .env file")
        print("   Get these values from Lab 4 deployment.")
        return

    print("🔌 Connecting to remote MCP server (Azure) via HostedMCPTool...")
    print(f"   URL: {mcp_remote_url[:60]}...")
    print("   ⏳ Note: First request may take 10-30s due to cold start...")
    print()

    # Azure Functions accepts auth via query parameter ?code=KEY
    mcp_url_with_key = f"{mcp_remote_url}?code={mcp_remote_key}"

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="RemoteMCPAssistant",
            instructions="""You are a helpful assistant with access to remote MCP tools deployed on Azure.
            
Available MCP Tools:
- hello_mcp: Health check
- analyze_data: Statistical analysis
- save_snippet/get_snippet/list_snippets: Code snippet storage

The tools are running on Azure Functions (serverless compute).""",
            tools=HostedMCPTool(
                name="lab4_remote_mcp",
                url=mcp_url_with_key,
            ),
        )

        print(f"✅ Agent created: {agent.name}")
        print(f"   MCP Server: lab4_remote_mcp (Azure, via HostedMCPTool)")
        print()

        async with agent:
            # Create thread for conversation
            thread = agent.get_new_thread()

            query = "Say hello using the hello_mcp tool"

            print("=" * 70)
            print(f"📤 User: {query}")
            print("=" * 70)

            # Run agent and handle tool approval requests
            result = await agent.run(query, thread=thread, store=True)

            # Auto-approve any tool calls (HostedMCPTool requires approval)
            while len(result.user_input_requests) > 0:
                print(f"\n🔧 Tool approval requested ({len(result.user_input_requests)} tools)")

                approval_responses = []
                for req in result.user_input_requests:
                    # Get function details if available
                    if hasattr(req, "function_call"):
                        func_name = req.function_call.name
                        func_args = req.function_call.arguments
                        print(
                            f"   ✅ Auto-approving: {func_name}({func_args[:50] if func_args else ''}...)"
                        )
                    else:
                        print(f"   ✅ Auto-approving tool call")

                    # Create approval response
                    approval_responses.append(
                        ChatMessage(
                            role="user",
                            contents=[req.create_response(True)],  # True = approved
                        )
                    )

                # Continue execution with approvals
                result = await agent.run(approval_responses, thread=thread, store=True)

            print(f"\n🤖 Assistant: {result}")


# Run the test
await test_remote_mcp_agent()

🔌 Connecting to remote MCP server (Azure) via HostedMCPTool...
   URL: https://func-api-4csy4jcpcp47y.azurewebsites.net/runtime/web...
   ⏳ Note: First request may take 10-30s due to cold start...

✅ Agent created: RemoteMCPAssistant
   MCP Server: lab4_remote_mcp (Azure, via HostedMCPTool)

📤 User: Say hello using the hello_mcp tool

🔧 Tool approval requested (1 tools)
   ✅ Auto-approving: hello_mcp({}...)

🤖 Assistant: Hello from MCP Server! 👋  
(Status: success, Lab: Lab 4 - MCP Server with Azure Functions)


## Step 5: User Approval Workflow with HostedMCPTool

This example demonstrates the **Microsoft-recommended pattern** for `HostedMCPTool` with interactive user approval.

### Why User Approval?

`HostedMCPTool` **always requires explicit approval** for tool calls. This is a security feature:
- Tools execute **server-side** (in Azure AI Agents service)
- You control which tools are allowed to run
- Prevents unintended actions on remote systems

### The Pattern

```python
async def handle_approvals_with_thread(query, agent, thread):
    result = await agent.run(query, thread=thread, store=True)
    
    while len(result.user_input_requests) > 0:
        approvals = []
        for req in result.user_input_requests:
            # Ask user (or auto-approve)
            approved = input("Approve? (y/n): ").lower() == "y"
            approvals.append(
                ChatMessage(role="user", contents=[req.create_response(approved)])
            )
        result = await agent.run(approvals, thread=thread, store=True)
    
    return result
```

This example uses **auto-approval** for the notebook, but shows the structure for interactive approval.

In [24]:
# Step 5: User Approval Workflow with HostedMCPTool
# Based on Microsoft's recommended pattern for Azure AI Agents with hosted MCP

from typing import Any
from agent_framework import AgentProtocol, AgentResponse, AgentThread, ChatMessage


async def handle_approvals_with_thread(
    query: str,
    agent: AgentProtocol,
    thread: AgentThread,
    auto_approve: bool = True,  # Set to False for interactive approval
) -> AgentResponse:
    """
    Handle tool approval workflow for HostedMCPTool.

    This is the Microsoft-recommended pattern for Azure AI Agents with hosted MCP.
    HostedMCPTool ALWAYS requires explicit approval for tool calls.

    Args:
        query: User's question
        agent: The AI agent with HostedMCPTool
        thread: Conversation thread for context
        auto_approve: If True, auto-approve all tool calls (for demos)

    Returns:
        Final agent response after all approvals processed
    """
    result = await agent.run(query, thread=thread, store=True)

    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []

        for user_input_needed in result.user_input_requests:
            func_name = user_input_needed.function_call.name
            func_args = user_input_needed.function_call.arguments

            print(f"\n🔧 Tool Call Request from {agent.name}:")
            print(f"   Function: {func_name}")
            print(f"   Arguments: {func_args}")

            if auto_approve:
                approved = True
                print("   ✅ Auto-approved")
            else:
                # Interactive approval (Microsoft pattern)
                user_approval = input("   Approve function call? (y/n): ")
                approved = user_approval.lower() == "y"
                print(f"   {'✅ Approved' if approved else '❌ Rejected'}")

            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(approved)],
                )
            )

        result = await agent.run(new_input, thread=thread, store=True)

    return result


async def test_hosted_mcp_with_approval():
    """
    Example showing HostedMCPTool with user approval workflow.

    This demonstrates the Microsoft-recommended pattern for Azure AI Agents
    with hosted MCP servers, including proper approval handling.
    """

    if not mcp_remote_url or not mcp_remote_key:
        print("❌ Remote MCP not configured! Run Lab 4 first.")
        return

    print("🔐 HostedMCPTool with User Approval Workflow")
    print("=" * 60)

    mcp_url_with_key = f"{mcp_remote_url}?code={mcp_remote_key}"

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="RemoteMCPAgent",
            instructions="""You are a helpful assistant with remote MCP tools.
Use the tools to help answer questions.""",
            tools=HostedMCPTool(
                name="lab4_remote_mcp",
                url=mcp_url_with_key,
            ),
        )

        # Must use agent as context manager for HostedMCPTool
        async with agent:
            thread = agent.get_new_thread()

            # First query
            query1 = "Say hello using the hello_mcp tool"
            print(f"\n📤 User: {query1}")
            print("-" * 60)
            result1 = await handle_approvals_with_thread(query1, agent, thread, auto_approve=True)
            print(f"🤖 {agent.name}: {result1}")

            print("\n" + "=" * 60 + "\n")

            # Second query (uses same thread for context)
            query2 = "Now analyze these numbers: [85, 92, 78, 95, 88]"
            print(f"📤 User: {query2}")
            print("-" * 60)
            result2 = await handle_approvals_with_thread(query2, agent, thread, auto_approve=True)
            print(f"🤖 {agent.name}: {result2}")


# Run the test
await test_hosted_mcp_with_approval()

🔐 HostedMCPTool with User Approval Workflow

📤 User: Say hello using the hello_mcp tool
------------------------------------------------------------

🔧 Tool Call Request from RemoteMCPAgent:
   Function: hello_mcp
   Arguments: {}
   ✅ Auto-approved
🤖 RemoteMCPAgent: Hello from MCP Server! 👋  
This is Lab 4 - MCP Server with Azure Functions.


📤 User: Now analyze these numbers: [85, 92, 78, 95, 88]
------------------------------------------------------------

🔧 Tool Call Request from RemoteMCPAgent:
   Function: analyze_data
   Arguments: {"values":"[85, 92, 78, 95, 88]"}
   ✅ Auto-approved
🤖 RemoteMCPAgent: Here are the statistics for the numbers `[85, 92, 78, 95, 88]`:

- **Count**: 5
- **Sum**: 438
- **Mean**: 87.6
- **Minimum**: 78
- **Maximum**: 95


## Step 6: Conversation with Thread History

Maintain conversation context across multiple queries using `AgentThread`.

This allows the agent to:
- Remember previous questions
- Reference earlier tool results
- Build on prior context

> 💡 **Note**: This example uses `MCPStreamableHTTPTool` (local) which auto-executes tools.

In [25]:
async def test_conversation_with_history():
    """Test multi-turn conversation with thread history."""

    print("💬 Multi-Turn Conversation with MCP Tools")
    print("=" * 60)

    async with (
        DefaultAzureCredential() as credential,
        MCPStreamableHTTPTool(
            name="lab4-mcp",
            url=mcp_local_url,
            load_prompts=False,  # Azure Functions MCP doesn't support prompts/list
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create agent WITHOUT tools - we pass them at run level
        agent = await provider.create_agent(
            name="ConversationAgent",
            instructions="""You are a helpful data analyst assistant.
You can analyze data and save results as snippets.
Remember the conversation context.""",
        )

        # Create a thread to maintain conversation history
        thread = agent.get_new_thread()
        print(f"📝 New conversation thread created\n")

        # Multi-turn conversation
        conversation = [
            "Analyze these test scores: [85, 92, 78, 95, 88]",
            "What was the mean score you just calculated?",
            "Save those results as a snippet called 'test_scores_analysis'",
            "What snippets do we have saved now?",
        ]

        for i, query in enumerate(conversation, 1):
            print(f"\n{'─' * 60}")
            print(f"📤 Turn {i}: {query}")
            print(f"{'─' * 60}")

            # Pass tools at run level (MCPStreamableHTTPTool pattern)
            result = await agent.run(query, thread=thread, tools=mcp_server, store=True)
            print(f"🤖 {result}")


# Run the test
await test_conversation_with_history()

💬 Multi-Turn Conversation with MCP Tools
📝 New conversation thread created


────────────────────────────────────────────────────────────
📤 Turn 1: Analyze these test scores: [85, 92, 78, 95, 88]
────────────────────────────────────────────────────────────
🤖 Here are the statistics for the test scores [85, 92, 78, 95, 88]:

- **Count**: 5 (total number of scores)
- **Sum**: 438 (total of all scores combined)
- **Mean**: 87.6 (average score)
- **Minimum**: 78 (lowest score)
- **Maximum**: 95 (highest score)

────────────────────────────────────────────────────────────
📤 Turn 2: What was the mean score you just calculated?
────────────────────────────────────────────────────────────
🤖 The mean score is **87.6**.

────────────────────────────────────────────────────────────
📤 Turn 3: Save those results as a snippet called 'test_scores_analysis'
────────────────────────────────────────────────────────────
🤖 The results have been successfully saved as a snippet called **'test_scores_analysi

## Step 7: Gradio Chat Interface 🎨

Build a complete chat interface that connects to your MCP server!

### Connection Modes

| Mode | Tool Class | Connection | Approval |
|------|-----------|------------|----------|
| 🏠 **Local** | `MCPStreamableHTTPTool` | Client → localhost:7071 | Auto-executes |
| ☁️ **Remote** | `HostedMCPTool` | Azure AI → Azure Function | Auto-approved |

### Key Implementation Details

**Local Mode:**
```python
# MCPStreamableHTTPTool - client-side, tools auto-execute
mcp = MCPStreamableHTTPTool(name="...", url=local_url)
result = await agent.run(query, tools=mcp)  # Tools execute automatically
```

**Remote Mode:**
```python
# HostedMCPTool - server-side, requires approval loop
agent = await provider.create_agent(tools=HostedMCPTool(name="...", url=remote_url))
async with agent:
    result = await agent.run(query)
    while result.user_input_requests:  # Must handle approvals!
        approvals = [ChatMessage(..., req.create_response(True)) for req in result.user_input_requests]
        result = await agent.run(approvals)
```

**Features:**
- Toggle between local and remote MCP servers
- Conversation history maintained across messages
- Auto-approval for remote MCP tool calls
- Example queries for easy testing

In [27]:
import gradio as gr
import nest_asyncio
from agent_framework import MCPStreamableHTTPTool, HostedMCPTool, ChatMessage
from agent_framework.azure import AzureAIAgentsProvider
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

# Enable nested event loops for Jupyter
nest_asyncio.apply()


# Session state for agent - supports both local and remote MCP
class MCPAgentSession:
    def __init__(self):
        self.agent = None
        self.thread = None
        self.provider = None
        self.credential = None
        self.mcp_server = None  # Only used for local (MCPStreamableHTTPTool)
        self.initialized = False
        self.use_remote = False
        self._agent_context = None  # For HostedMCPTool context manager

    async def initialize(self, use_remote: bool = False):
        """Initialize agent with local or remote MCP server.

        Local: Uses MCPStreamableHTTPTool (client-side, auto-executes)
        Remote: Uses HostedMCPTool (server-side, requires approval)
        """
        # Cleanup previous session if switching servers
        if self.initialized and self.use_remote != use_remote:
            await self.cleanup()

        if not self.initialized:
            self.use_remote = use_remote

            if use_remote and mcp_remote_url and mcp_remote_key:
                # REMOTE: Use HostedMCPTool with AzureCliCredential
                # Server-side connection, requires approval for tool calls
                self.credential = AzureCliCredential()
                self.provider = AzureAIAgentsProvider(credential=self.credential)
                await self.provider.__aenter__()

                mcp_url_with_key = f"{mcp_remote_url}?code={mcp_remote_key}"

                self.agent = await self.provider.create_agent(
                    name="MCPChatAssistant",
                    instructions="""You are a helpful assistant with MCP tools (Azure remote server).

Available MCP Tools:
📡 hello_mcp - Health check to verify MCP connectivity
📊 analyze_data - Analyze numbers (pass as JSON array like [1,2,3])
💾 save_snippet - Save code with a name
📖 get_snippet - Retrieve saved code
📋 list_snippets - List all saved snippets

Be helpful and use tools when appropriate.""",
                    tools=HostedMCPTool(
                        name="lab4_remote_mcp",
                        url=mcp_url_with_key,
                    ),
                )
                # Enter agent context for HostedMCPTool
                self._agent_context = self.agent
                await self._agent_context.__aenter__()

                self.thread = self.agent.get_new_thread()
                self.initialized = True
                print(f"✅ Agent initialized with Azure (remote) MCP server via HostedMCPTool")

            else:
                # LOCAL: Use MCPStreamableHTTPTool (client-side, auto-executes)
                self.credential = DefaultAzureCredential()
                self.provider = AzureAIAgentsProvider(credential=self.credential)
                await self.provider.__aenter__()

                self.mcp_server = MCPStreamableHTTPTool(
                    name="lab4_local_mcp",
                    url=mcp_local_url,
                    load_prompts=False,
                )
                await self.mcp_server.__aenter__()

                # Create agent WITHOUT tools - pass them at run level for MCPStreamableHTTPTool
                self.agent = await self.provider.create_agent(
                    name="MCPChatAssistant",
                    instructions="""You are a helpful assistant with MCP tools (Local server).

Available MCP Tools:
📡 hello_mcp - Health check to verify MCP connectivity
📊 analyze_data - Analyze numbers (pass as JSON array like [1,2,3])
💾 save_snippet - Save code with a name
📖 get_snippet - Retrieve saved code
📋 list_snippets - List all saved snippets

Be helpful and use tools when appropriate.""",
                )
                self.thread = self.agent.get_new_thread()
                self.initialized = True
                print(f"✅ Agent initialized with Local MCP server via MCPStreamableHTTPTool")

    async def chat(self, message: str, use_remote: bool = False) -> str:
        """Process chat message with MCP-enabled agent."""
        try:
            await self.initialize(use_remote)

            if use_remote:
                # Remote (HostedMCPTool): Requires approval handling
                result = await self.agent.run(message, thread=self.thread, store=True)

                # Auto-approve tool calls (HostedMCPTool requires this)
                while len(result.user_input_requests) > 0:
                    approval_responses = []
                    for req in result.user_input_requests:
                        if hasattr(req, "function_call"):
                            print(f"   🔧 Auto-approving: {req.function_call.name}")
                        approval_responses.append(
                            ChatMessage(
                                role="user",
                                contents=[req.create_response(True)],
                            )
                        )
                    result = await self.agent.run(
                        approval_responses, thread=self.thread, store=True
                    )

                return str(result)
            else:
                # Local (MCPStreamableHTTPTool): Auto-executes tools
                result = await self.agent.run(
                    message,
                    thread=self.thread,
                    tools=self.mcp_server,
                    store=True,
                )
                return str(result)

        except Exception as e:
            import traceback

            traceback.print_exc()
            return f"❌ Error: {str(e)}"

    async def cleanup(self):
        """Cleanup resources."""
        if self._agent_context:
            await self._agent_context.__aexit__(None, None, None)
            self._agent_context = None
        if self.mcp_server:
            await self.mcp_server.__aexit__(None, None, None)
            self.mcp_server = None
        if self.provider:
            await self.provider.__aexit__(None, None, None)
            self.provider = None
        if self.credential:
            await self.credential.__aexit__(None, None, None)
            self.credential = None
        self.agent = None
        self.thread = None
        self.initialized = False

    def reset(self):
        """Reset conversation thread."""
        if self.agent:
            self.thread = self.agent.get_new_thread()


# Create session
session = MCPAgentSession()


# Gradio response function
async def respond(message: str, history: list, use_remote: bool):
    """Handle chat messages."""
    if not message.strip():
        return history, ""

    bot_response = await session.chat(message, use_remote)

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": bot_response})
    return history, ""


def clear_chat():
    """Clear chat and reset thread."""
    session.reset()
    return []


# Build Gradio interface
with gr.Blocks(title="MCP Agent Chat - Lab 5") as demo:
    gr.Markdown("""
    # 🔌 MCP Agent Chat (Lab 5)
    
    Chat with an AI agent connected to your **MCP server from Lab 4**!
    
    **Connection Modes:**
    - 🏠 **Local** (`MCPStreamableHTTPTool`): Client-side connection, tools auto-execute
    - ☁️ **Remote** (`HostedMCPTool`): Server-side connection via Azure AI, auto-approved
    
    **Available Tools:**
    - 📡 `hello_mcp` — Health check
    - 📊 `analyze_data` — Statistical analysis (use JSON arrays: [1,2,3])
    - 💾 `save_snippet` / `get_snippet` / `list_snippets` — Code storage
    """)

    with gr.Row():
        use_remote = gr.Checkbox(
            label="☁️ Use Remote (Azure) MCP Server",
            value=False,
            info="Local uses MCPStreamableHTTPTool; Remote uses HostedMCPTool",
        )

    chatbot = gr.Chatbot(
        height=450,
        placeholder="Ask me anything! I have MCP tools for data analysis and snippet storage.",
    )

    msg = gr.Textbox(
        placeholder="Type your message... (e.g., 'Analyze [85, 92, 78, 95, 88]')",
        label="Your Message",
        lines=2,
    )

    with gr.Row():
        submit = gr.Button("Send 📤", variant="primary")
        clear = gr.Button("Clear 🗑️")

    gr.Examples(
        examples=[
            "Say hello to verify MCP connectivity",
            "Analyze these test scores: [85, 92, 78, 95, 88, 76, 91, 83]",
            "Save this Python code as 'fibonacci': def fib(n): return n if n < 2 else fib(n-1) + fib(n-2)",
            "List all saved snippets",
            "Get the 'fibonacci' snippet",
        ],
        inputs=msg,
    )

    # Event handlers
    msg.submit(respond, [msg, chatbot, use_remote], [chatbot, msg])
    submit.click(respond, [msg, chatbot, use_remote], [chatbot, msg])
    clear.click(clear_chat, outputs=[chatbot])

# Launch
print("🚀 Launching MCP Agent Chat...")
print(f"   🏠 Local MCP: {mcp_local_url}")
if mcp_remote_url:
    print(f"   ☁️  Remote MCP: {mcp_remote_url[:50]}...")
print("\n📝 Toggle checkbox to switch between Local and Remote MCP servers")
demo.launch(share=False, height=700)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1178a4650>


🚀 Launching MCP Agent Chat...
   🏠 Local MCP: http://localhost:7071/runtime/webhooks/mcp
   ☁️  Remote MCP: https://func-api-4csy4jcpcp47y.azurewebsites.net/r...

📝 Toggle checkbox to switch between Local and Remote MCP servers
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


✅ Agent initialized with Local MCP server via MCPStreamableHTTPTool


## Step 8: Combining MCP with Local Tools

You can combine **MCP tools** with **local Python functions** for a hybrid approach.

This is useful when:
- Some tools need to be MCP (shared across systems)
- Some tools are specific to this agent (local functions)

> 💡 **Note**: This example uses `MCPStreamableHTTPTool` (local) which auto-executes tools.

In [28]:
from typing import Annotated
from pydantic import Field
from datetime import datetime


# Local function tool (not MCP)
def get_current_time() -> str:
    """Get the current date and time."""
    now = datetime.now()
    return f"📅 Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"


def calculate_grade(
    score: Annotated[float, Field(description="The score achieved")],
    max_score: Annotated[float, Field(description="Maximum possible score")],
) -> str:
    """Calculate letter grade from score."""
    percentage = (score / max_score) * 100

    if percentage >= 93:
        letter = "A"
    elif percentage >= 90:
        letter = "A-"
    elif percentage >= 87:
        letter = "B+"
    elif percentage >= 83:
        letter = "B"
    elif percentage >= 80:
        letter = "B-"
    elif percentage >= 77:
        letter = "C+"
    elif percentage >= 73:
        letter = "C"
    elif percentage >= 70:
        letter = "C-"
    elif percentage >= 60:
        letter = "D"
    else:
        letter = "F"

    return f"📝 Score: {score}/{max_score} = {percentage:.1f}% → Grade: {letter}"


async def test_hybrid_tools():
    """Test agent with both MCP tools and local functions."""

    print("🔀 Hybrid Agent: MCP Tools + Local Functions")
    print("=" * 60)

    async with (
        DefaultAzureCredential() as credential,
        MCPStreamableHTTPTool(
            name="lab4-mcp",
            url=mcp_local_url,
            load_prompts=False,  # Azure Functions MCP doesn't support prompts/list
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create agent with both local functions and MCP tools
        agent = await provider.create_agent(
            name="HybridAssistant",
            instructions="""You are a helpful academic assistant with both local and MCP tools.

Local Tools:
- get_current_time: Get current date/time
- calculate_grade: Convert score to letter grade

MCP Tools (from Lab 4):
- analyze_data: Statistical analysis of numbers
- save_snippet/get_snippet/list_snippets: Code storage

Use the appropriate tool for each task.""",
            tools=[
                get_current_time,  # Local function
                calculate_grade,  # Local function
                mcp_server,  # MCP tools (multiple tools from server)
            ],
        )

        print(f"✅ Hybrid agent created with local + MCP tools\n")

        # Test query using multiple tool types
        query = """Please help me with the following:
        1. What time is it now?
        2. I got 87 out of 100 on my exam - what's my letter grade?
        3. Analyze these quiz scores: [85, 92, 78, 95, 88]
        """

        print(f"📤 User: {query}")
        print("=" * 60)
        result = await agent.run(query)
        print(f"\n🤖 Assistant: {result}")


# Run the test
await test_hybrid_tools()

🔀 Hybrid Agent: MCP Tools + Local Functions
✅ Hybrid agent created with local + MCP tools

📤 User: Please help me with the following:
        1. What time is it now?
        2. I got 87 out of 100 on my exam - what's my letter grade?
        3. Analyze these quiz scores: [85, 92, 78, 95, 88]
        

🤖 Assistant: Here are your results:

1. **Current Time:** January 21, 2026, 14:38 (2:38 PM).
2. **Letter Grade:** You scored 87 out of 100, which is 87% → Grade: **B+**.
3. **Quiz Scores Analysis:**  
   - **Count:** 5 scores  
   - **Sum:** 438  
   - **Mean:** 87.6  
   - **Minimum Score:** 78  
   - **Maximum Score:** 95  


## Summary

In this lab, you learned how to integrate **MCP servers** with **Azure AI Agents**!

### Steps Completed

| Step | Topic | Tool Used |
|------|-------|-----------|
| 1 | Setup & Environment | - |
| 2 | Install Packages | - |
| 3 | Local MCP Server | `MCPStreamableHTTPTool` |
| 4 | Remote MCP Server | `HostedMCPTool` |
| 5 | User Approval Workflow | `HostedMCPTool` + approval loop |
| 6 | Thread History | `MCPStreamableHTTPTool` |
| 7 | Gradio Chat UI | Both tools |
| 8 | Hybrid (MCP + Local) | `MCPStreamableHTTPTool` |

### Key Concepts

| Concept | Description |
|---------|-------------|
| **MCPStreamableHTTPTool** | Client-side MCP connection, auto-executes tools |
| **HostedMCPTool** | Server-side MCP via Azure AI, **requires approval** |
| **User Approval** | `user_input_requests` + `create_response(True/False)` |
| **Thread History** | `AgentThread` maintains conversation context |
| **Hybrid Tools** | Combine MCP + local functions in one agent |

### MCP Integration Patterns

```python
# Pattern 1: Local MCP (auto-executes)
async with MCPStreamableHTTPTool(url="http://localhost:7071/...") as mcp:
    result = await agent.run(query, tools=mcp)

# Pattern 2: Remote MCP (requires approval)
agent = await provider.create_agent(tools=HostedMCPTool(url="https://..."))
async with agent:
    result = await agent.run(query, thread=thread, store=True)
    while result.user_input_requests:  # Handle approvals!
        approvals = [ChatMessage(role="user", contents=[req.create_response(True)]) 
                     for req in result.user_input_requests]
        result = await agent.run(approvals, thread=thread, store=True)

# Pattern 3: Hybrid (MCP + local functions)
agent = await provider.create_agent(tools=[local_func, mcp_server])
```

### Connection Settings

| Server Type | URL | Authentication |
|-------------|-----|----------------|
| Local | `http://localhost:7071/runtime/webhooks/mcp` | None |
| Azure | `https://func-xxx.azurewebsites.net/runtime/webhooks/mcp?code=KEY` | Query param |

### What's Next?

- **Lab 6**: Multi-agent systems with specialized agents
- **Production**: Deploy your MCP-enabled agent as an API
- **Extend**: Add more MCP tools to your Lab 4 server

---

**🎉 Congratulations!** You've built an AI agent that can use MCP tools from both local and cloud servers!